<a href="https://colab.research.google.com/github/BashC0de/Blindness-Detection/blob/main/Bill_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import csv
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict


In [ ]:
def parse_date(date_str):# This Converts a date string in YYYY-MM-DD format into a Python datetime
    return datetime.strptime(date_str, "%Y-%m-%d")

def get_month_range(target_month):#This takes a target month (e.g., "2025-05") which i have taken from csv file and returns the first and last date of that month.
    start = datetime.strptime(target_month, "%Y-%m")
    end = (start.replace(day=28) + timedelta(days=4)).replace(day=1) - timedelta(days=1)
    return start, end

def days_overlap(start1, end1, start2, end2):#Calculates how many days overlap between two date ranges. Essential for prorating item usage.
    latest_start = max(start1, start2)
    earliest_end = min(end1, end2)
    delta = (earliest_end - latest_start).days + 1
    return max(0, delta), latest_start, earliest_end

def normalize_value(val):#This helps in consistent numeric formatting by stripping and converting strings to float
    if isinstance(val, str):
        return float(val.strip())
    return float(val)

def generate_monthly_bill(item_list: list, target_month: str, target_item_code: str) -> dict:#  Main function to generate the prorated monthly bill for all items.
    month_start, month_end = get_month_range(target_month)
    days_in_month = (month_end - month_start).days + 1

    grouped_items = defaultdict(lambda: {"qty": 0, "amount": 0.0, "billing_period": None})

    for item in item_list:
        item_start = parse_date(item["start_date"])
        item_stop = parse_date(item["stop_date"])

        # These are checking below if the item is active in the target month
        overlap_days, overlap_start, overlap_end = days_overlap(item_start, item_stop, month_start, month_end)
        if overlap_days == 0:
            continue

        item_code = item["item_code"]
        rate = normalize_value(item["rate"])
        qty = int(item["qty"])
        daily_rate = rate * qty / days_in_month
        prorated_amount = round(daily_rate * overlap_days, 2)

        # These keys includes all biling range
        billing_period = f"{overlap_start.strftime('%Y-%m-%d')} to {overlap_end.strftime('%Y-%m-%d')}"
        group_key = (item_code, rate, billing_period)

        grouped_items[group_key]["qty"] += qty
        grouped_items[group_key]["amount"] += prorated_amount
        grouped_items[group_key]["billing_period"] = billing_period

    # Preparing  final output
    line_items = []
    total_revenue = 0.0

    for (item_code, rate, _), values in grouped_items.items():
        amount = round(values["amount"], 2)
        total_revenue += amount
        line_items.append({
            "item_code": item_code,
            "rate": rate,
            "qty": values["qty"],
            "amount": amount,
            "billing_period": values["billing_period"]
        })

    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2)
    }
def read_items_from_csv(file_path):
    items = []
    with open(file_path, mode="r", newline='', encoding="utf-8", errors="replace") as file:
        reader = csv.DictReader(file)
        for row in reader:
            items.append({
                "item_code": row["item_code"],
                "rate": row["rate"],
                "qty": int(row["qty"]),
                "start_date": row["start_date"],
                "stop_date": row["stop_date"]
            })
    return items


In [ ]:
if __name__ == "__main__":
    file_path = "Bill.csv"#this bill is the sample bill that i have created
    start_date_str = "2025-05-01"
    end_date_str = "2025-05-30"
    target_month = "2025-04"
    target_item_code="B001" #target code from the csv file to get the bill

    # Parsing  dates from string to datetime

    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    item_list = read_items_from_csv(file_path)
    bill = generate_monthly_bill(item_list, target_month, target_item_code)
    print(json.dumps(bill, indent=2))#this will give the proper structure of bill

{
  "line_items": [
    {
      "item_code": "C003",
      "rate": 10.75,
      "qty": 2,
      "amount": 7.88,
      "billing_period": "2025-04-20 to 2025-04-30"
    },
    {
      "item_code": "G007",
      "rate": 18.75,
      "qty": 4,
      "amount": 15.0,
      "billing_period": "2025-04-25 to 2025-04-30"
    }
  ],
  "total_revenue": 22.88
}
